## Web crawler


import modules


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait


import time
import numpy as np
import pandas as pd
import pickle

initialize args


In [3]:
service = Service()
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values': {'cookies': 2, 'images': 2, 'javascript': 2, 
                            'plugins': 2, 'popups': 2, 'geolocation': 2, 
                            'notifications': 2, 'auto_select_certificate': 2, 'fullscreen': 2, 
                            'mouselock': 2, 'mixed_script': 2, 'media_stream': 2, 
                            'media_stream_mic': 2, 'media_stream_camera': 2, 'protocol_handlers': 2, 
                            'ppapi_broker': 2, 'automatic_downloads': 2, 'midi_sysex': 2, 
                            'push_messaging': 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop': 2, 
                            'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement': 2, 
                            'durable_storage': 2}}
options.add_experimental_option('prefs', prefs)
options.page_load_strategy = 'eager'

options.add_argument('--headless')  # 啟動Headless 無頭



extract publisher, creator, year from raw data


In [4]:
def process_rawData(creatorRaw, publisherRaw):
       creator = creatorRaw.removesuffix(', 著')
       split = publisherRaw.split(',')
       year = split[-1]
       year = year.split('[')[0]
       publisher = split[0].split(':')[-1].replace('\n', ' ')
       publisher = publisher.removeprefix(' ')
       
       return creator, publisher, year
       

crawler function


In [6]:
def crawl_book(bookTitle):
       driver = webdriver.Chrome(service=service, options=options)
       wait = WebDriverWait(driver, timeout=10, poll_frequency=1)
       baseUrl = 'https://metadata.ncl.edu.tw/blstkmc/blstkm#tudorkmtop'
       driver.get(baseUrl)
       
       # set search configs
       x1 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table[1]/tbody/tr[2]/td/table/tbody/tr[2]/td[1]/select'
       select = Select(wait.until(
                     EC.presence_of_element_located((By.XPATH, x1))
                     ))
       select.select_by_value('n_TI')
       x2 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table[1]/tbody/tr[2]/td/table/tbody/tr[2]/td[2]/select'
       select = Select(driver.find_element(By.XPATH, x2))
       select.select_by_value('C')
       # enter book name
       x3 = '//*[@id="hotname"]'
       query = driver.find_element(By.XPATH, x3)
       query.clear()
       query.send_keys(bookTitle)
       # press search
       x4 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table[1]/tbody/tr[2]/td/table/tbody/tr[5]/td/input[1]'
       searchBtn = wait.until(
                     EC.presence_of_element_located((By.XPATH, x4))
                     )
       searchBtn.click()
       
       
       try:
              # sort by year , ascending
              x6 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr[4]/td[1]/table/tbody/tr/td[3]/table/tbody/tr/td[2]/select'
              select = Select( wait.until(
                     EC.presence_of_element_located((By.XPATH, x6))
              ))
              select.select_by_value('YR')
              x7 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr[4]/td[1]/table/tbody/tr/td[3]/table/tbody/tr/td[3]/select'
              select = Select( wait.until(
                     EC.presence_of_element_located((By.XPATH, x7))
              ))
              select.select_by_value('0')
              x8 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr[4]/td[1]/table/tbody/tr/td[2]/input'
              sortBtn =  wait.until(
                     EC.presence_of_element_located((By.XPATH, x8))
              )
              sortBtn.click()
              
              xCreator = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr[5]/td/table/tbody/tr[2]/td[3]'
              xPublisher  = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr[5]/td/table/tbody/tr[2]/td[4]'
              creatorRaw =   wait.until(
                     EC.presence_of_element_located((By.XPATH, xCreator))
              ).text
              publisherRaw =  wait.until(
                     EC.presence_of_element_located((By.XPATH, xPublisher))
              ).text
              creator, publisher, year = process_rawData(creatorRaw, publisherRaw)
       except:
              creator, publisher, year = 'error', 'error', 'error'
              print('error', bookTitle)  
       driver.close()  
       
       return pd.Series( [bookTitle, creator, publisher, year], index=['title','creator','publisher','year'])


test one book 

In [8]:
crawl_book('三體')

title           三體
creator        劉慈欣
publisher      貓頭鷹
year          2023
dtype: object

In [9]:
df = pd.read_csv('書籍.csv')
df = df.dropna().reset_index()
print(df['書名'])

0               龍鱗焰火
1               龍族秘錄
2              黑暗的左手
3               黃海童話
4        非理性時代 牛頓加農砲
           ...      
496        [簡]納米魔幻兵團
497    [簡]現代中國科幻文學主潮
498     黃禍 3/3黃禍反撲世界
499     黃禍 1/3大陸內戰爆發
500    黃禍  2/3台灣生死存亡
Name: 書名, Length: 501, dtype: object


In [15]:
errorIndex = []
start = 100
checkpoint = 50
new_df = pd.DataFrame()

In [31]:
for i in range(start, start+checkpoint):
      series = crawl_book(df['書名'][i])
      if(series['creator']== 'error'):
            errorIndex.append(i) 
      new_df = pd.concat([new_df, series],axis=1)
      if(i%10 == 0):
            if(i!=0): 
                  print()
            print(f'done{i}/{start+checkpoint}', end = '')      
      print('.', end = '')

new_df.to_csv(f'./csv/{start}-{start+checkpoint}.csv')
start+=checkpoint


error Star Trek the Next Generation: Perchance to Dream

done400/450.error Star Trek the Next Generation: Metamorphosis
.error Star Trek the Next Generation: Doomsday World
.error Star Trek the Next Generation: Contamination
.error Star Trek the Next Generation: Chains of Command
.error Star Trek the Next Generation Immortal Coil
.error Star Trek the Next Generation Gateway
.error Star Trek S.C.E. Book Two
.error Star Trek S.C.E. Book One
.error STAR TREK Roleplaying Game Player`s Guide
.error Star Trek Roleplaying Game Narrator's Guide

done410/450.error Star Trek ds9
.error Star Trek Deep Space 9: Avatar Book Two
.error Star Trek Deep Space 9: Avatar Book One
.error Star Trek Deep Space 9 Section 31
.error Star Trek Cookbook
.....
done420/450.error Shadow of the Hegemon
...error Science Fiction SF: The Illustrated Encyclopedia
.error Science Fiction of the 20th Century
.

ElementClickInterceptedException: Message: element click intercepted: Element <input type="image" name="_IMG_檢索" alt="__檢索" src="/blstkm/img/search_btn.png" class="button" border="0" title="檢索"> is not clickable at point (430, 306). Other element would receive the click: <div id="ajax_listOfOptions" style="top: 199px; left: 415px;"></div>
  (Session info: chrome-headless-shell=127.0.6533.101)
Stacktrace:
0   chromedriver                        0x00000001048d9024 cxxbridge1$str$ptr + 1887276
1   chromedriver                        0x00000001048d1700 cxxbridge1$str$ptr + 1856264
2   chromedriver                        0x00000001044e082c cxxbridge1$string$len + 88524
3   chromedriver                        0x000000010452a020 cxxbridge1$string$len + 389568
4   chromedriver                        0x000000010452864c cxxbridge1$string$len + 382956
5   chromedriver                        0x0000000104526878 cxxbridge1$string$len + 375320
6   chromedriver                        0x0000000104525dc

In [30]:
errorIndex=[]
new_df=pd.DataFrame()

save var

In [32]:
with open(f'./var/var{start-checkpoint}-{start}.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([start, checkpoint, errorIndex, new_df], f)

load var

In [16]:
with open(f'./var/var{start-checkpoint}-{start}.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    start,checkpoint,errorIndex,new_df_ = pickle.load(f)